This notebook provides the framework for doing the batch transform for hourly call volume predictions using NFORS data. The cells at the beginning of the notebook are all required for running the code, but the end of the notebook contains the code I used for creating the datasets for reference.

In [3]:
#If running in local mode, local to True. Otherwise, set it to false
local = True

# Getting the Sagemaker role

In [2]:
import sagemaker
from sagemaker import get_execution_role

# S3 prefix
prefix = 'hourly_call_volume'


if local == True:
    sagemaker_session = sagemaker.LocalSession()
    role = 'arn:aws:iam::445861113736:role/service-role/AmazonSageMaker-ExecutionRole-20190903T114521'
else:
    sagemaker_session = sagemaker.Session()
    role = get_execution_role()

## Create SageMaker Scikit Estimator <a class="anchor" id="create_sklearn_estimator"></a>

To run our Scikit-learn training script on SageMaker, we construct a `sagemaker.sklearn.estimator.sklearn` estimator, which accepts several constructor arguments:

* __entry_point__: The path to the Python script SageMaker runs for training and prediction.
* __role__: Role ARN
* __train_instance_type__ *(optional)*: The type of SageMaker instances for training. __Note__: Because Scikit-learn does not natively support GPU training, Sagemaker Scikit-learn does not currently support training on GPU instance types.
* __sagemaker_session__ *(optional)*: The session used to train on Sagemaker.
* __hyperparameters__ *(optional)*: A dictionary passed to the train function as hyperparameters.

To see the code for the SKLearn Estimator, see here: https://github.com/aws/sagemaker-python-sdk/tree/master/src/sagemaker/sklearn

In [4]:
from sagemaker.sklearn.estimator import SKLearn

script_path = 'hourly_call_prediction.py'
sklearn = SKLearn(
    entry_point=script_path,
    train_instance_type="ml.c4.xlarge",
    role=role,
    sagemaker_session=sagemaker_session,
    hyperparameters={'n_estimators': 1000})

## Train SKLearn Estimator on call volume data

In [5]:
#The data should already be saved to the ./data directory
WORK_DIRECTORY = 'data'
train_input = sagemaker_session.upload_data(WORK_DIRECTORY, key_prefix="{}/{}".format(prefix, WORK_DIRECTORY) )

#Training the model
sklearn.fit({'train': train_input})

Creating tmpqh5cc6pp_algo-1-usz56_1 ... 
Attaching to tmpqh5cc6pp_algo-1-usz56_12mdone
algo-1-usz56_1  | 2020-01-22 20:26:37,663 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
algo-1-usz56_1  | 2020-01-22 20:26:37,667 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-usz56_1  | 2020-01-22 20:26:37,684 sagemaker_sklearn_container.training INFO     Invoking user training script.
algo-1-usz56_1  | 2020-01-22 20:26:38,004 sagemaker-containers INFO     Module hourly_call_prediction does not provide a setup.py. 
algo-1-usz56_1  | Generating setup.py
algo-1-usz56_1  | 2020-01-22 20:26:38,004 sagemaker-containers INFO     Generating setup.cfg
algo-1-usz56_1  | 2020-01-22 20:26:38,004 sagemaker-containers INFO     Generating MANIFEST.in
algo-1-usz56_1  | 2020-01-22 20:26:38,004 sagemaker-containers INFO     Installing module with the following command:
algo-1-usz56_1  | /miniconda3/bin/python -m pip install . 
algo-1-usz56

tmpqh5cc6pp_algo-1-usz56_1 exited with code 0
Aborting on container exit...
===== Job Complete =====


## Batch Transform <a class="anchor" id="batch_transform"></a>
We can also use the trained model for asynchronous batch inference on S3 data using SageMaker Batch Transform.

In [15]:
# Define a SKLearn Transformer from the trained SKLearn Estimator
transformer = sklearn.transformer(instance_count=1, instance_type='ml.m4.xlarge')

### Run Transform Job <a class="anchor" id="run_transform_job"></a>
Using the Transformer, run a transform job on the S3 input data.

In [17]:
#Again, the test data should already be saved to the prediction_data directory
WORK_DIRECTORY = 'prediction_data/test_data'
batch_input_s3 = sagemaker_session.upload_data(WORK_DIRECTORY, key_prefix="{}/{}".format(prefix, WORK_DIRECTORY) )

# Start a transform job and wait for it to finish
transformer.transform(batch_input_s3, content_type='text/csv',  join_source='Input')
print('Waiting for transform job: ' + transformer.latest_transform_job.job_name)
transformer.wait()

Attaching to tmpeogb29mz_algo-1-srq34_1
algo-1-srq34_1  | Processing /opt/ml/code
algo-1-srq34_1  | Building wheels for collected packages: hourly-call-prediction
algo-1-srq34_1  |   Building wheel for hourly-call-prediction (setup.py) ... done
algo-1-srq34_1  |   Created wheel for hourly-call-prediction: filename=hourly_call_prediction-1.0.0-py2.py3-none-any.whl size=7279 sha256=1e700c9be7609fc3d7ff415458286d605735e9b7ec01d7dbea8c5990889b57de
algo-1-srq34_1  |   Stored in directory: /tmp/pip-ephem-wheel-cache-v7s9p4o3/wheels/35/24/16/37574d11bf9bde50616c67372a334f94fa8356bc7164af8ca3
algo-1-srq34_1  | Successfully built hourly-call-prediction
algo-1-srq34_1  | Installing collected packages: hourly-call-prediction
algo-1-srq34_1  | Successfully installed hourly-call-prediction-1.0.0
algo-1-srq34_1  | /miniconda3/lib/python3.7/site-packages/sklearn/externals/joblib/externals/cloudpickle/cloudpickle.py:47: DeprecationWarning: the imp module is deprecated in favour of importlib; see the m

algo-1-srq34_1  | [2020-01-22 20:33:06 +0000] [4115] [INFO] Booting worker with pid: 4115
algo-1-srq34_1  | [2020-01-22 20:33:06 +0000] [4116] [INFO] Booting worker with pid: 4116
algo-1-srq34_1  | 172.18.0.1 - - [22/Jan/2020:20:33:06 +0000] "GET /execution-parameters HTTP/1.1" 404 232 "-" "-"
algo-1-srq34_1  | [2020-01-22 20:33:06 +0000] [4244] [INFO] Booting worker with pid: 4244
algo-1-srq34_1  | [2020-01-22 20:33:07 +0000] [4245] [INFO] Booting worker with pid: 4245
algo-1-srq34_1  | 2020-01-22 20:33:07,244 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
algo-1-srq34_1  | /miniconda3/lib/python3.7/site-packages/sklearn/externals/joblib/externals/cloudpickle/cloudpickle.py:47: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
algo-1-srq34_1  |   import imp
algo-1-srq34_1  | 172.18.0.1 - - [22/Jan/2020:20:33:08 +0000] "POST /invocations HTTP/1.1" 200 6662 "-" "-"
Gracefully stoppin

In [9]:
with open('./prediction_data/test_data', 'r') as f:
    data = f.read()

In [11]:
from io import StringIO
import pandas as pd
features = pd.read_csv(StringIO(data),dtype={'hour': object, 'fire_department.firecares_id': object})

In [12]:
features = pd.get_dummies(features)
features = features.reindex(sorted(features.columns),axis=1)

In [13]:
features

,description.day_of_week_Friday,description.day_of_week_Monday,description.day_of_week_Saturday,description.day_of_week_Sunday,description.day_of_week_Thursday,description.day_of_week_Tuesday,description.day_of_week_Wednesday,fire_department.firecares_id_79592,fire_department.firecares_id_93345,hour_00,...,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23
0,0,1,0,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331,0,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
332,0,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
333,0,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
334,0,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


# Exporting prediction results as json

In [ ]:
import json
import pandas as pd

# Download the output data from S3 to local filesystem
batch_output = transformer.output_path
!mkdir -p batch_data/output
!aws s3 cp --recursive $batch_output/ batch_data/output/

#Loading the predictions and appending them to 
with open('./batch_data/output/test_data.json.out', 'r') as file:
    pred = file.read()
pred = pred.strip('][').split(', ')
pred = [float(i) for i in pred]


#Loading the test dataframe so we can append the results to it
with open('./prediction_data/test_data.json') as datafile:
    data = json.load(datafile)
results = pd.io.json.json_normalize(data['prediction_data'])
results['call_volume'] = pred


#Saving it as a json
jsondata = {}
jsondata['model_name'] = 'calls_by_hour'
jsondata['model_version'] = 1.0
jsondata['results'] = results.to_dict(orient='records')

with open('./prediction_data/results.json', 'w') as outfile:
    json.dump(jsondata,outfile)


# Example plots

In [ ]:
import matplotlib.pyplot as plt
day = 'Friday'
fc_id = '79592'




subset = results[ (results['fire_department.firecares_id'] == fc_id) & (results['description.day_of_week'] == day)]
subset.plot.bar('hour', 'call_volume')

# Creating the training data and uploading to s3
This cell only works locally because it depends on elasticsearch.


In [ ]:
#Importing libraries required for performing the query
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
from elasticsearch_dsl import Q
import pandas as pd
import json

#Setting up the query
es = Elasticsearch()
s = Search(using=es,index='*-fire-incident-*')
s = s.source(['description.event_opened',
                     'description.day_of_week',
                    'NFPA.type',
                     'fire_department.firecares_id'])


#Performing the query
q = Q("match",fire_department__firecares_id =  '79592') | Q("match",fire_department__firecares_id =  '93345')
results = s.query(q)

#Converting query results to a pandas dataframe
df = pd.DataFrame((d.to_dict() for d in tqdm_notebook(results.scan())))
json_struct = json.loads(df.to_json(orient="records"))

df = pd.io.json.json_normalize(json_struct)

#Converting date
df['date'] = df['description.event_opened'].apply(lambda x: x[:10])
df['month'] = df.apply(lambda x: x['date'][5:7], axis=1)
df['hour'] = df['description.event_opened'].apply(lambda x: x[11:13])


#Converting df dates to datetime objects
df['date'] = df.apply(lambda x: datetime.datetime.strptime(x['date'],'%Y-%m-%d'),axis=1)
# df['date'] = df.apply(lambda x: datetime.datetime.strptime(x['date'],'%Y-%m-%d'),axis=1)

#It's convenient to serialize (pickle) the dataframe because it's faster to load it rather than re-create it.
df.to_pickle('query_results')

#Hourly is a dataframe aggregated grouped by the day, hour, and department
hourly = df[['fire_department.firecares_id', 'date','description.day_of_week', 'hour']].groupby(['fire_department.firecares_id', 'date','description.day_of_week', 'hour']).aggregate(len).reset_index()
hourly = hourly.rename(columns={0: 'calls'})

#Formatting the hourly dataframe into a json
jsondata = {}
jsondata['model_name'] = 'calls_by_hour'
jsondata['model_version'] = 1.0
jsondata['prediction_data'] = hourly.drop('date',axis=1).to_dict(orient='records')

#Saving the json to the data directory
with open('./data/training_data.json', 'w') as outfile:
    json.dump(jsondata, outfile)
    


# Creating the test dataset
This involves generating a dataframe with every possible combination of department, day of week, and hour of the day. The size of this dataset is 7x24xn, where n is the number of departments included for predictions.

Also, this cell requires that the training set has already been created.

In [ ]:
#Make every combination of departments, days of week, and hour
from itertools import product
import json
import pandas as pd



#load the training dataset
with open('./data/training_data.json') as data_file:
    data = json.load(data_file)
hourly = pd.io.json.json_normalize(data['prediction_data'])

#Getting the list of ever department that shows up in the hourly dataframe
dep_list = hourly['fire_department.firecares_id'].unique()
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
hours = [str(i).zfill(2) for i in range(24)]

#Creating the dataframe of all possible combinations
test_df = pd.DataFrame(list(product(dep_list, days, hours)), columns=['fire_department.firecares_id', 'description.day_of_week', 'hour'])

#Saving it as a csv
test_df.to_csv('./prediction_data/test_data', index=False)




In [ ]:
hourly